# EfficientNetV2 学習・評価・TFLite 化スクリプト

# 学習・評価の準備

## 認証

Google Cloud Storage にデータセットをアップロードした場合は、次のセルを実行して認証処理を行います。

> ️📘ノート
>
> * データセットを Google Drive にアップロードした場合は、この手順は不要です。

In [ ]:
## EfficientNet-V2 TF-Hub 版
## ver. 20220118

!mkdir -p /content
!gcloud auth application-default login

## Google Drive のマウント

Google Drive にデータセットをアップロードした場合は、次のセルを実行して Google Drive を仮想マシンにマウントします。

Google Drive は `/content/drive/MyDrive` にマウントされます。

> ️📘ノート
>
> * データセットを Google Cloud Storage にアップロードした場合は、この手順は不要です。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 学習・評価の設定

In [ ]:
#@markdown ## データセットのパス
#@markdown - Google Cloud Storage にアップロードした場合
#@markdown   - ディレクトリの URL を指定
#@markdown   - 例: `gs://mybucket/path/to/dataset_dir`
#@markdown - Google Drive にアップロードした場合
#@markdown   - ディレクトリパスを指定
#@markdown   - 例: `/content/drive/MyDrive/dataset`
GCS_PATH = "/content/drive/MyDrive/dataset" #@param {type: "string"}

#@markdown ---

#@markdown ## 大規模データセットかどうか
#@markdown
#@markdown 非常に大きなデータセット（画像10万枚以上）では、メモリエラーが発生することがあります。
#@markdown
#@markdown このチェックボックスを選択すると、メモリエラーを回避する処理を実施します。
#
# True のときは OOM 回避のために
# - 学習時、エポックごとの validation を省略する
# - 推論時、遅いが OOM にならない手法を使う
HUGE_DATASET = False #@param {type: "boolean"}

#@markdown ---

#@markdown ## 学習画像枚数
#@markdown 
#@markdown `train` ディレクトリに含まれる画像数の合計を入力します。
NUM_TRAINING_IMAGES = 792 #@param {type: "number"}
#@markdown ## 推論画像枚数
#@markdown 
#@markdown `validation` ディレクトリに含まれる画像数の合計を入力します。
NUM_VALIDATION_IMAGES = 247 #@param {type: "number"}
NUM_TEST_IMAGES = NUM_VALIDATION_IMAGES

#@markdown ---

# TF-Hub のモデルのパス
CORE_LAYER_PATH = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2'

# 画像サイズ [H, W]
IMAGE_SIZE = [512, 512]
# リサイズ方式 ("RESIZE": 拡大・縮小, "PAD": パディング)
RESIZE_METHOD = "RESIZE"
#@markdown ## バッチサイズ
#@markdown 
#@markdown 1ステップ（1回のモデル更新）で使用する画像の数を指定します。
#@markdown 
#@markdown バッチサイズが大きいほど学習速度が向上しますが、大きすぎるとメモリ不足エラーになります。
#@markdown 
#@markdown 目安は `256` (TPU)、`4` (GPU) です。
BATCH_SIZE = 4 #@param {type: "number"}
# エポック数
EPOCHS = 30

# 90度ずつ回転する水増し (bool)
AUG_ROTATE = True
# RandAug (bool)
AUG_RANDAUG = False
AUG_RANDAUG_N = 2 # レイヤ数
AUG_RANDAUG_M = 5 # マグニチュード
# > 0.0 ならば Mixup を適用
AUG_MIXUP = 0.0
# 1.2倍 / 0.8倍の拡大・縮小 (bool)
AUG_ZOOM = False

#@markdown ---
#@markdown
#@markdown ## 転移学習元モデルのファイルパス
#@markdown
#@markdown 転移学習（学習済みのモデルをもとに、別のモデルを学習する）の設定です。
#@markdown
#@markdown 転移学習しないときは `None` を指定します。
#@markdown
#@markdown 転移学習を行うときは、転移学習元のモデルファイルのパスを指定します。
#@markdown
#@markdown * モデルは、データとインデックスの2ファイルに分かれています。  
#@markdown   ファイル名の共通する部分を、ファイルパスとして指定します。  
#@markdown
#@markdown   例: 下記のファイルから転移学習するとき
#@markdown      ```
#@markdown      ckpt-30.data-00000-of-00001`
#@markdown      ckpt-30.index`
#@markdown      ```
#@markdown   → `"gs://bucket/path/to/ckpt-30"` のように指定します。
#@markdown * Google Cloud Storage のモデルを参照する場合
#@markdown   * モデルのURLを指定
#@markdown   * 例: `"gs://bucket/my_training/ckpt-30"`
#@markdown * Google Drive 上のモデルを参照する場合
#@markdown   * モデルのファイルパスを指定
#@markdown   * 例: `"/content/drive/MyDrive/my_training/ckpt-30"`
TRANSFER_LEARNING_FROM = None #@param {type: "raw"}
# head 層のみ学習するエポック数 (HEAD_EPOCHS <= EPOCHS)
HEAD_EPOCHS = 5

# 学習率（1cycle）
LR_RAMPUP_EPOCHS = 4
LR_SUSTAIN_EPOCHS = 0
LR_START = 0.00001
LR_MAX_BASE = 0.00005   # 実際の Max 学習率は(ノード数)倍になる
LR_MIN = 0.00001
LR_EXP_DECAY = .8

#@markdown ---

#@markdown ## 学習済みモデルの保存先
#@markdown - Google Cloud Storage にモデルを保存する場合
#@markdown   - Google Cloud Storage のディレクトリ URL を指定
#@markdown   - 例: `gs://mybucket/path/to/dataset_dir`
#@markdown   - アクセラレータを TPU にセットする場合は、Cloud Storage に保存する必要があります
#@markdown - Google Drive にモデルを保存する場合
#@markdown   - ディレクトリパスを指定
#@markdown   - 例: `/content/drive/MyDrive/model`
LOG_DIR = "/content/drive/MyDrive/custom_ai/model.tflite" #@param {type: "string"}


In [ ]:
import math, re, os
import json
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics \
    import f1_score, precision_score, recall_score, confusion_matrix
from functools import partial

print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
synset_id2label_path = os.path.join(GCS_PATH, "synset_id2label.json") 

with tf.io.gfile.GFile(synset_id2label_path, "r") as f:
    synset_id2label = json.loads(f.read())

class_id_sorted = sorted(synset_id2label, key=lambda k: int(k))
CLASSES = [synset_id2label[k] for k in class_id_sorted]

In [ ]:
### TFRecord のクラスを読み替えるかどうか
#REMAP_CLASSES = False
#
### TFRecord のクラス読み替え用辞書
###  - key に TFRecord 作成時のクラス名
###  - value に読み替え後のクラス名
### を書くと、TFRecord のクラス名を読み替えて学習できます
#remap_dict = {
#    "1_fruit_azamiuma": "1_fruit_azamiuma",
#    "1_fruit_healthy": "1_fruit_healthy",
#    "1_fruit_otabakoga": "1_fruit_otabakoga",
#    "1_leaf_aburamushi": "aburamushi",
#    "1_leaf_azamiuma": "leaf_azamiuma",
#    "1_leaf_hamoguribae": "leaf_hamoguribae",
#    "1_leaf_hasumonyoto": "leaf_hasumonyoto",
#    "1_leaf_healthy": "leaf_healthy",
#    "1_leaf_konajirami": "konajirami",
#    "1_leaf_tomatosabidani": "1_leaf_tomatosabidani",
#}
#
### 読み替え用のリストを定義
### Remaps labels in TFRecord to the labels given.
### Note that labels start from 1, not 0.
### e.g.) read_tfrecord(..., relabel=[1,1,2])
###   -> The model interprets labels 1 and 2 from TFRecord as label 1
###                           label 3 from TFRecord as label 2
#if REMAP_CLASSES:
#    relabel_list = []
#    new_id2label = list(sorted(set(remap_dict.values())))
#
#    for k in sorted(remap_dict.keys()):
#        v = remap_dict[k]
#        new_idx = new_id2label.index(v) + 1
#        print(f'Old class "{k}" is mapped to "{v}" ({new_idx})')
#        relabel_list.append(new_idx)
#
#    ## CLASSES を各所で使っている。再定義する
#    CLASSES = new_id2label
#else:
#    print("Remapping will not be applied.")
relabel_list = None

In [ ]:
## 計算ノード（TPU/GPU/CPU）の判別
try: # detect TPUs
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: # detect GPUs
    #strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines
    #raise Exception("I find your lack of TPUs disappointing.")
print("Number of accelerators: ", strategy.num_replicas_in_sync)

## 入力値バリデーション
assert HEAD_EPOCHS <= EPOCHS, f'Hey, training the head until epoch {HEAD_EPOCHS}, '\
    f'then all layers until epoch {EPOCHS}, makes no sense.'
allowed_resize_methods = ("RESIZE", "PAD")
assert RESIZE_METHOD in allowed_resize_methods, \
    f"Invalid RESIZE_METHOD: {RESIZE_METHOD}. Choose from {allowed_resize_methods}."

training_dirpath = os.path.join(GCS_PATH, "train-*")
TRAINING_FILENAMES = tf.io.gfile.glob(training_dirpath)
validation_dirpath = os.path.join(GCS_PATH, "validation-*")
VALIDATION_FILENAMES = tf.io.gfile.glob(validation_dirpath)
TEST_FILENAMES = VALIDATION_FILENAMES

def lrfn(epoch):
    LR_MAX = LR_MAX_BASE * strategy.num_replicas_in_sync
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

rng = [i for i in range(EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

# データセット

In [ ]:
# データセット
## Data augmentation
def random_flip(image):
    '''画像を水平・垂直反転して返します
    '''
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    return image


def random_rot90(image):
    '''Randomly rotates the image by (n / 2) * pi radian. (n=0,1,2,3)
    画像を90度/180度/270度だけ回転して返します
    '''
    k = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    image = tf.cond(k >= 1,
                    true_fn=lambda: tf.image.rot90(image, k=k),
                    false_fn=lambda: image)
    return image


def sample_beta_distribution(size, concentration_0=0.2, concentration_1=0.2):
    gamma_1_sample = tf.random.gamma(shape=[size], alpha=concentration_1)
    gamma_2_sample = tf.random.gamma(shape=[size], alpha=concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)


def mix_up(ds_one, ds_two, alpha=0.2):
    # Unpack two datasets
    images_one, labels_one = ds_one
    images_two, labels_two = ds_two
    batch_size = tf.shape(images_one)[0]

    # Sample lambda and reshape it to do the mixup
    l = sample_beta_distribution(batch_size, alpha, alpha)
    x_l = tf.reshape(l, (batch_size, 1, 1, 1))
    y_l = tf.reshape(l, (batch_size, 1))

    # Perform mixup on both images and labels by combining a pair of images/labels
    # (one from each dataset) into one image/label
    images = images_one * x_l + images_two * (1 - x_l)
    labels = labels_one * y_l + labels_two * (1 - y_l)
    return (images, labels)


def randaug(image, num_layers=2, magnitude=15):
    '''RandAugment を適用します
    (本家 EffNet-V2 から流用した TensorFlow 純正の実装)
    Args:
        image: `tf.Tensor`, 値域 [0,1]
        num_layers: `int`, レイヤ数
        magnitude: `int`, 画像効果の強度
    Returns:
        `tf.Tensor` 形式の画像, 値域 [0,1]
    '''
    print("Applying RandAugment.")
    image *= 255
    input_image_type = image.dtype
    image = tf.clip_by_value(image, 0.0, 255.0)
    image = tf.cast(image, dtype=tf.uint8)  # float では???
    image = autoaugment.distort_image(
        image, aug_name='randaug', ra_num_layers=num_layers, 
        ra_magnitude=magnitude)
    image = tf.cast(image, dtype=input_image_type)
    image /= 255

    return image


def zoom(image):
    """Zooms the image by factor of 1.2."""
    shape = tf.shape(image)
    ## Zoom by (about) 1.2x
    image = tf.image.central_crop(image, 0.8)
    ## Then crop
    image = tf.image.resize(image, shape[:2])

    return image


def mooz(image):
    """Moozes the image by factor of 0.8."""
    shape = tf.shape(image)
    s0 = tf.cast(shape[0], tf.float32)
    s1 = tf.cast(shape[1], tf.float32)
    ## Increases the image size by 1.2x with pad
    image = tf.image.pad_to_bounding_box(
        image,
        offset_height=tf.cast(s0*0.1, tf.int32),
        offset_width=tf.cast(s1*0.1, tf.int32),
        target_height=tf.cast(s0*1.2, tf.int32),
        target_width=tf.cast(s1*1.2, tf.int32))
    ## Then resizes to the original shape
    image = tf.image.resize(image, shape[:2])

    return image


def random_zoom_mooz(image):
    '''画像を1/3の確率で拡大切り抜き、縮小して、またはそのまま返します
    拡大・縮小の中心点は、画像の中心です
    Args:
        image: `tf.Tensor`
    Returns:
        `tf.Tensor` 形式の画像、幅と高さは image と同じ
    '''
    k = tf.random.uniform(shape=[], minval=0, maxval=3, dtype=tf.int32)
    image = tf.cond(k == 1,
                    true_fn=lambda: zoom(image),
                    false_fn=lambda: image)
    image = tf.cond(k == 2,
                    true_fn=lambda: mooz(image),
                    false_fn=lambda: image)

    return image

In [ ]:
def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)

    ## RESIZE_METHOD に応じて、サイズ合わせの方法（パディング/リサイズ）を切り替え
    if RESIZE_METHOD == "PAD":
        image = tf.image.resize_with_pad(
            image, IMAGE_SIZE[0], IMAGE_SIZE[1], 
            method=tf.image.ResizeMethod.BICUBIC)
    else:
        image = tf.image.resize(
            image, IMAGE_SIZE, method=tf.image.ResizeMethod.BICUBIC)

    return image


def read_tfrecord(example, labeled, one_hot=False, relabel=None):
    '''Parses Google AutoML-style TFRecord dataset.
    Args:
        example
        labeled
        one_hot
        relabel: `list`. Remaps labels in TFRecord to the labels given.
            Note that labels start from 1, not 0.
            e.g.) read_tfrecord(..., relabel=[1,1,2])
              -> The model interprets labels 1 and 2 from TFRecord as label 1
                                      label 3 from TFRecord as label 2
    '''
    tfrecord_format = (
        {
            "image/encoded": tf.io.FixedLenFeature([], tf.string),
            "image/class/label": tf.io.FixedLenFeature([], tf.int64),
        }
        if labeled
        else {"image": tf.io.FixedLenFeature([], tf.string),}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image/encoded"])
    image /= 255
    if labeled:
        label = tf.cast(example["image/class/label"], tf.int64)
        if relabel is not None:
            label -= 1  # Since (image/class/label == 1) maps to relabel[0] 
            relabel_t = tf.constant(relabel, tf.float32)
            label = tf.gather(relabel_t, label)
        label -= 1
        if one_hot:
            print("One hot is active!")
            label = tf.cast(label, tf.uint8)
            label = tf.one_hot(label, len(CLASSES))
            label = tf.cast(label, tf.float32)
        return image, label
    return image


def load_dataset(
        filenames, labeled=True, ordered=False, one_hot=False, relabel=None):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False  
            # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames)  
        # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order)
        # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(
            read_tfrecord, labeled=labeled, one_hot=one_hot, relabel=relabel),
        num_parallel_calls=AUTO)
        # returns a dataset of (image, label) pairs if labeled=True 
        # or just images if labeled=False

    return dataset


def _get_training_dataset(
    aug_rotate=False, one_hot=False, aug_randaug=False, aug_zoom=False,
    randaug_num_layers=2, randaug_magnitude=15, relabel=None):
    '''Obtain the training datset.
    Args:
      aug_rotate: `bool`. If true, performs random rotation and flip.
      one_hot: `bool`, True ならばワンホット符号化する（Mixup用）
      aug_randaug: `bool`. True ならば、RandAugment を適用する
      aug_zoom: `bool`. True ならば、拡大・縮小水増しを適用する
      randaug_num_layers: `int`, RandAugment のレイヤ数
      randaug_magnitude: `int`, RandAugment の画像効果の強度
      relabel: `dict` or None, `dict` ならば TFRecord のラベルを読み替える
    '''
    dataset = load_dataset(
        TRAINING_FILENAMES, labeled=True, one_hot=one_hot, relabel=relabel)

    if aug_rotate:
        dataset = dataset.map(
            lambda x, y: (random_flip(x), y),
            num_parallel_calls=AUTO)
        dataset = dataset.map(
            lambda x, y: (random_rot90(x), y), 
            num_parallel_calls=AUTO)

    if aug_randaug:
        dataset = dataset.map(
            lambda x, y: (
                randaug(
                    x, num_layers=randaug_num_layers, 
                    magnitude=randaug_magnitude),
                y), 
            num_parallel_calls=AUTO)

    if aug_zoom:
        dataset = dataset.map(
            lambda x, y: (random_zoom_mooz(x), y),
            num_parallel_calls=AUTO)

    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)

    return dataset


def get_training_dataset(aug_mixup=0.0, **kwargs):
    '''Wraps _get_training_dataset to add mixup functionality.
    Args:
      aug_mixup  `float`. If > 0.0, performs mixup augmentation.
      **kwargs   See the definition of _get_training_dataset().'''
    if aug_mixup == 0.0:
        print('Mixup is disabled.')
        return _get_training_dataset(**kwargs)
    else:
        print('Mixup is enabled.')
        dataset_1 = _get_training_dataset(one_hot=True, **kwargs)
        dataset_2 = _get_training_dataset(one_hot=True, **kwargs)
        train_ds = tf.data.Dataset.zip((dataset_1, dataset_2))
        train_ds = train_ds.map(
            lambda ds1, ds2: mix_up(ds1, ds2, alpha=aug_mixup),
            num_parallel_calls=AUTO)
        return train_ds

    
def get_validation_dataset(ordered=False, one_hot=False, relabel=None):
    dataset = load_dataset(
        VALIDATION_FILENAMES, labeled=True, ordered=ordered, one_hot=one_hot,
        relabel=relabel)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset


def get_test_dataset(ordered=False, relabel=None):
    dataset = load_dataset(
        TEST_FILENAMES, labeled=False, ordered=ordered, relabel=relabel)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

NUM_TEST_IMAGES = NUM_VALIDATION_IMAGES
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
VALIDATION_STEPS = -(-NUM_VALIDATION_IMAGES // BATCH_SIZE) # The "-(-//)" trick rounds up instead of down :-)
TEST_STEPS = -(-NUM_TEST_IMAGES // BATCH_SIZE)             # The "-(-//)" trick rounds up instead of down :-)
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

# モデル定義

In [ ]:
## Custom callbacks, from the original EffNet-V2
## 
class ReusableBackupAndRestore(tf.keras.callbacks.BackupAndRestore):
  """A BackupAndRestore callback that can be used across multiple model.fit()s."""

  def on_train_end(self, logs=None):
    # don't delete the backup, so it can be used for future model.fit()s
    pass

In [ ]:
with strategy.scope():
    # TF>=2.4: load models from TF Hub directly to the TPU
    load_locally = tf.saved_model.LoadOptions(
        experimental_io_device='/job:localhost')
    pretrained_model = hub.KerasLayer(
        CORE_LAYER_PATH, trainable=True, input_shape=[*IMAGE_SIZE, 3], 
        load_options=load_locally)
    model = tf.keras.Sequential([
        # the expected image format for all TFHub image models is float32 in
        # [0,1) range
        tf.keras.layers.Lambda(
            lambda data: tf.image.convert_image_dtype(data, tf.float32), 
            input_shape=[*IMAGE_SIZE, 3]),
        pretrained_model,
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ])

    # Optionally make the core layer untrainable (train only the head).
    if HEAD_EPOCHS >= 1:
        model.layers[1].trainable = False

    ## Optionally initialize the model with pretrained data.
    if TRANSFER_LEARNING_FROM is not None:
        ## Remove the last layer, load weights, then attach a fresh head.
        model.pop()
        model.load_weights(TRANSFER_LEARNING_FROM)
        model.add(tf.keras.layers.Dense(len(CLASSES), activation='softmax'))


## 損失関数: Mixupあり（One-hot）の場合は categorical_crossentropy
##          Mixupなしの場合は sparse_categorical_crossentropy
loss_fn = 'categorical_crossentropy' if AUG_MIXUP > 0 \
          else 'sparse_categorical_crossentropy'
print(f"Loss function: {loss_fn}")

model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy'],
    steps_per_execution=16,
)
model.summary()

# 学習

In [ ]:
train_ds = get_training_dataset(
    aug_rotate=AUG_ROTATE, aug_mixup=AUG_MIXUP, aug_randaug=AUG_RANDAUG, 
    aug_zoom=AUG_ZOOM, randaug_num_layers=AUG_RANDAUG_N, 
    randaug_magnitude=AUG_RANDAUG_M, relabel=relabel_list)

callbacks = [
    lr_callback,
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(LOG_DIR, 'ckpt-{epoch:d}'), verbose=1,
        save_weights_only=True),
    tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, update_freq=100),
    ReusableBackupAndRestore(backup_dir=LOG_DIR),
]

## 大規模データセットのときは、学習時の validation を省略して OOM 回避
if HUGE_DATASET:
    args_validation = {}
else:
    args_validation = {
        "validation_data": get_validation_dataset(one_hot=(AUG_MIXUP>0.0)),
        "validation_steps": VALIDATION_STEPS,
    }

## HEAD_EPOCHS が 1 以上のときは
##  - 先に head 層を HEAD_EPOCHS まで学習する
##  - 全レイヤを学習可能にして、再度学習する
if HEAD_EPOCHS >= 1:
    history = model.fit(
        train_ds, steps_per_epoch=STEPS_PER_EPOCH, epochs=HEAD_EPOCHS, 
        callbacks=callbacks, verbose=2, **args_validation)
    model.layers[1].trainable = True
    model.summary()
    model.compile(
        optimizer='adam',
        loss=loss_fn,
        metrics=['accuracy'],
        steps_per_execution=16,
        )

history = model.fit(
    train_ds, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, 
    callbacks=callbacks, verbose=2, **args_validation)

# 学習済みモデルの評価

In [ ]:
final_ckpt_path = os.path.join(LOG_DIR, f'ckpt-{EPOCHS}')
model.load_weights(final_ckpt_path)

if not HUGE_DATASET:
    cmdataset = get_validation_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and labels, order matters.
    images_ds = cmdataset.map(lambda image, label: image)
    labels_ds = cmdataset.map(lambda image, label: label).unbatch()
    cm_correct_labels = next(iter(labels_ds.batch(NUM_VALIDATION_IMAGES))).numpy() # get everything as one batch
    cm_probabilities = model.predict(images_ds, steps=VALIDATION_STEPS)
    cm_predictions = np.argmax(cm_probabilities, axis=-1)
    cm_predictions = cm_predictions[:NUM_VALIDATION_IMAGES] 
        # Lest some validation data are counted twice
    #print("Correct   labels: ", cm_correct_labels.shape, cm_correct_labels)
    #print("Predicted labels: ", cm_predictions.shape, cm_predictions)

if HUGE_DATASET:
    import keras.backend as K
    from tqdm import tqdm

    ## 巨大（画像数万枚、画像サイズ1408）な tf.Dataset で推論すると OOM になる
    ## - GPU ではホストのRAMを使い果たしてクラッシュする
    ## - TPU では `Socket closed` エラーが起こる
    ## 関連？: https://github.com/keras-team/keras/issues/13118
    ##
    ## 対策として、tf.Dataset を model に渡さず、
    ## ホストVM側でデータセットを少しずつ読みだして (`model(x)`)、TPU に推論させる
    ## 動作速度が遅くなるが、推論はできる
    ## 注意: GPU では OOM になる
    BS = 64
    TAKE = int((NUM_VALIDATION_IMAGES // BS) * 1)
    data_len = BS * TAKE

    cm_correct_labels = np.empty((data_len), dtype=int)    # Stores GTs
    cm_predictions = np.empty((data_len), dtype=int)       # Stores preds

    cmdataset = tf.data.TFRecordDataset(VALIDATION_FILENAMES)
    dataset = cmdataset.map(
        partial(
            read_tfrecord, labeled=True, one_hot=False, relabel=relabel_list),
        num_parallel_calls=AUTO)
    
    for i, batch in tqdm(
            enumerate(dataset.shuffle(100).batch(BS).take(TAKE)), total=TAKE):
        images_batch, labels_batch = batch
        images_batch = tf.squeeze(images_batch)
            # (bs, 1, h, w, c) -> (bs, h, w, c)
        prob_y = model(images_batch, training=False) 
            # since model.predict() causes OOM
        pred_y = tf.math.argmax(prob_y, axis=-1)
        labels_batch = tf.cast(labels_batch, tf.int8)
        labels_batch = tf.squeeze(labels_batch)

        # 結果を保存
        idx_from = i * BS
        idx_to = idx_from + BS
        cm_correct_labels[idx_from:idx_to] = labels_batch
        cm_predictions[idx_from:idx_to] = pred_y

In [ ]:
## F値などの算出
cmat = confusion_matrix(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)))
score = f1_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
precision = precision_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
recall = recall_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
cmat = (cmat.T / cmat.sum(axis=1)).T # normalized
#display_confusion_matrix(cmat, score, precision, recall)
print('f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(score, precision, recall))

In [ ]:
## 混同行列をCSV形式で出力
import pandas as pd
cmat = confusion_matrix(
    cm_correct_labels, cm_predictions, 
    labels=range(len(np.unique(cm_correct_labels))))
df = pd.DataFrame(cmat, index=CLASSES, columns=CLASSES)
print(df.to_csv())

In [ ]:
## Top-1精度の算出
top_1_acc = (cm_correct_labels == cm_predictions).sum() / len(cm_correct_labels)
print(f'top_1_accuracy: {top_1_acc * 100:.1f}%')

# 学習済みモデルの変換

学習済みの画像識別モデルを、エッジ端末で動作可能な形式（TFLite）に変換します。

変換により、識別性能はやや低下します。

> ⚠️注意⚠️
> 
> アクセラレータの設定が TPU のときに変換を試みると、`converter.convert()` の部分でエラーになります：
>
> ```
> ValueError: Device /job:worker/replica:0/task:0/device:CPU:0 is not found
> ```
>
> アクセラレータの設定が TPU になっている場合は、**アクセラレータをTPU以外に設定してVMを再起動し、学習・評価以外の各セルを再実行してから**下記のコードを実行してください。


In [ ]:
#@markdown ## TFLite モデルの書き出し先
#@markdown
#@markdown 書き出すファイルのパスを指定します。
#@markdown
#@markdown Google Drive をマウントしている場合は、`/content/drive/MyDrive/` 以下のファイルパスを指定することで、Google Drive にモデルを保存できます。
#@markdown
#@markdown 例: `/content/drive/MyDrive/model.tflite` (Google Drive に書き出す場合)
EXPORT_TFLITE_TO = "/content/drive/MyDrive/model.tflite" #@param {type: "string"}

In [ ]:
final_ckpt_path = os.path.join(LOG_DIR, f'ckpt-{EPOCHS}')
print(f"Loading {final_ckpt_path}")
model.load_weights(final_ckpt_path)

In [ ]:
import itertools
from pathlib import Path

## パラメータの設定
optimize_lite_model = True
quantize_lite_model = True
num_calibration_examples = 100
tflite_export_path = Path(EXPORT_TFLITE_TO)

representative_dataset = None

## validation データを使用して、軽量化モデルを最適化する
if optimize_lite_model and num_calibration_examples:
  # Use a bounded number of training examples without labels for calibration.
  # TFLiteConverter expects a list of input tensors, each with batch size 1.
  representative_dataset = lambda: itertools.islice(
      ([image[None, ...]] for batch, _ in get_validation_dataset() for image in batch),
      num_calibration_examples)

## TFLite モデルの変換
#converter = tf.lite.TFLiteConverter.from_saved_model(LOG_DIR)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
if optimize_lite_model:
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
if quantize_lite_model:
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
if representative_dataset:  # This is optional, see above.
    converter.representative_dataset = representative_dataset
lite_model_content = converter.convert() # 3～4分かかる

## 作成した TFLite モデルの書き出し
tflite_export_path.parent.mkdir(parents=True, exist_ok=True)
with open(tflite_export_path, "wb") as f:
    f.write(lite_model_content)
    print("Wrote %sTFLite model of %d bytes." %
        ("optimized " if optimize_lite_model else "", len(lite_model_content)))

In [ ]:
## TFLite モデルのシグネチャ（入出力テンソル）のデータ確認
#interpreter = tf.lite.Interpreter(model_content=lite_model_content)
## This little helper wraps the TF Lite interpreter as a numpy-to-numpy function.
#
#input_details = interpreter.get_input_details()
#output_details = interpreter.get_output_details()
#
#print(input_details)
#print(output_details)
#
#def lite_model(images):
#  interpreter.allocate_tensors()
#  interpreter.set_tensor(interpreter.get_input_details()[0]['index'], images)
#  interpreter.invoke()
#  return interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

## LICENSE

Copyright 2021 MARTIN GÖRNER

Copyright 2022 Sailaja Raja and Ronil Modi

Copyright 2021 Google Inc. All rights reserved.

Copyright 2021 The TensorFlow Hub Authors. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.


Original files:

* https://www.kaggle.com/code/rajasailaja/test-tpu
* https://www.kaggle.com/code/mgornergoogle/efficientnetb7-on-100-flowers
* https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/d617988b1cc7de73ec11a6015fd1983a6925af5e/03_image_models/03b_finetune_experiment_lr_decay_xception_flowers104.ipynb
* https://github.com/tensorflow/hub/blob/12b09be736893efd11a0a54d85e0e01516f44442/examples/colab/tf2_image_retraining.ipynb
* https://www.tensorflow.org/hub/tutorials/tf2_image_retraining

Major changes from the original:

* Combined the above files to create a train-evaluation-quantization pipleline of EfficientV2 models